<a href="https://colab.research.google.com/github/bernardocamps/imersao-ia-labbot/blob/main/LabBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar e importar o Google Generative AI

In [255]:
!pip install -q -U google-generativeai
import google.generativeai as genai

# Configurar a API Key

In [256]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Listar os modelos disponíveis

In [257]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

2024-05-11 01:43:40.792 200 GET /v1beta/models?pageSize=50&%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2257.31ms


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


# Configurar os parâmetros

In [258]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

# Configurar as opções de segurança

In [259]:
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

# Inicializar o modelo

In [260]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

# Criar chat:

In [261]:
chat = model.start_chat(history=[])

# Iniciar chat com o LabBot:

In [268]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import contextlib
import io
from urllib.parse import quote

# Estilos CSS
style = """
<style>
.widget-box {
    border-radius: 15px;
    box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.2);
    padding: 20px;
    margin-bottom: 20px;
}
.output-widget {
    padding: 15px;
    border-radius: 8px;
}
.input-widget input[type="text"] {
    width: 100%;
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
}
button {
    border-radius: 10px !important;
}
p {
    line-height: 1.6;
}
a {
    color: #007bff;
    text-decoration: none;
}
a:hover {
    text-decoration: underline;
}
@media (max-width: 768px) {
    .widget-box {
        width: 90%;
        margin: 0 auto;
    }
    .input-widget input[type="text"], .output-widget {
        font-size: 16px;
    }
}
</style>
"""
display(widgets.HTML(style))

# Widgets principais
output_widget = widgets.Output(layout={'padding': '10px'})
input_widget = widgets.Text(description="Exames:", layout={'width': '400px'})
city_widget = widgets.Text(description="Cidade:", layout={'width': '300px'})
button_widget = widgets.Button(description="Pesquisar", button_style='info', layout={'width': '90px'})

# Layout
display(widgets.VBox([
    widgets.HTML("<h2>LabBot: Seu parceiro na busca pelo especialista ideal!</h2>"),
    widgets.HTML("<p>Exames alterados? 🤔 Conte para mim quais são e onde você está, que eu te ajudo a encontrar o médico correto para avaliar o seu caso:</p>"),
    widgets.HBox([input_widget, city_widget, button_widget]),
    output_widget
]))

# Função para enviar a mensagem
def enviar_mensagem(_):
    exames = input_widget.value
    cidade = city_widget.value
    input_widget.value = ""
    city_widget.value = ""

    if exames.lower() == "fim":
        with output_widget:
            clear_output()
            print("👋 Até logo!")
        return

    with output_widget:
        with contextlib.redirect_stdout(io.StringIO()):
            response = chat.send_message(f"Para os exames alterados '{exames}', qual a especialidade médica mais indicada para avaliação? Responda apenas o nome do especialista recomendado. Não coloque o resultado entre ** **")

        clear_output()
        print(f"Médico recomendado: {response.text}")

        # Gera link para o Google Maps
        especialidade_encoded = quote(response.text)
        cidade_encoded = quote(cidade)
        maps_link = f"https://www.google.com/maps/search/{especialidade_encoded}+{cidade_encoded}"
        display(widgets.HTML(f'<a href="{maps_link}" target="_blank">Veja no Google Maps</a> 🔍'))

# Eventos
button_widget.on_click(enviar_mensagem)


HTML(value='\n<style>\n.widget-box {\n    border-radius: 15px;\n    box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.2)…